# Data Augmentation - Experimento

Data Augmentation é uma estratégia bastante utilizada para impulsionar treinamento de modelos. Se baseando em diversas transformações nos dados, as abordagens de Data Augmentation conseguem multiplicar os seus dados mantendo os mesmos rótulos. Exemplos dessas transformações em dados de imagens são rotações, translações, mudança de coloração, e etc.

A implementação desse componente foi feita utilizando a biblioteca [torchvision](https://pytorch.org/vision/stable/index.html).

## Declaração de Classe para Predições em Tempo Real

A tarefa de implantação cria um serviço REST para predições em tempo-real.<br>
Para isso você deve criar uma classe `Model` que implementa o método `predict`.

In [ ]:
%%writefile Model.py
from typing import List, Iterable, Dict, Union

import numpy as np
import cv2
import torchvision.transforms as T
import joblib
from PIL import Image
import io
from io import StringIO

class Model:
    
    def __init__(self):
        self.loaded = False
        
    def load(self):
        # Carrega artefatos: estimador, etc
        artifacts = joblib.load("/tmp/data/data_augmentation.joblib")
        self.artifacts = artifacts["data_augmentation_parameters"]
        
        self.parameters = [ self.artifacts["augmentation_rate"],
                            self.artifacts["horizontal_flip"],
                            self.artifacts["vertical_flip"],
                            self.artifacts["crop"],
                            self.artifacts["color_jitter"],
                            self.artifacts["perspective"],
                            self.artifacts["rotate"] ]

        self.augmentation_rate = self.parameters[0]
        
        # Load Model

        self.jitter = T.ColorJitter(brightness=.5, hue=.3)
        self.perspective_transformer = T.RandomPerspective(distortion_scale=0.6, p=1.0)
        self.rotater = T.RandomRotation(degrees=(0, 180))
        self.hflip = T.RandomHorizontalFlip()
        self.vflip = T.RandomVerticalFlip() 
        
        #self.transformations = [ self.jitter, self.perspective_transformer, self.rotater, self.crop, self.hflip, self.vflip ]
        
        self.loaded = True
        
    def predict(self, X, feature_names, meta=None):

        if not self.loaded:
            self.load()
            
        # Check if data is a bytes
        if isinstance(X, bytes):
            im_bytes = X # Get image bytes
        
        # If not, should be a list or ndarray
        else:
            # Garantee is a ndarray
            X = np.array(X)
            
            # Seek for extra dimension
            if len(X.shape) == 2:
                im_bytes = X[0,0] # Get image bytes
            
            else:
                im_bytes = X[0] # Get image bytes
        
        # Preprocess img bytes to img_arr
        im_arr = np.frombuffer(im_bytes, dtype=np.uint8)
        img = cv2.imdecode(im_arr, flags=cv2.IMREAD_COLOR)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(img) # convert to PIL image

        width, height = img.size
        crop_size = (int(width * 0.8),int(height * 0.8))  
        self.crop = T.RandomCrop(crop_size)

        transformed_images = []
        if self.parameters[1]:
            transformed_images += [self.hflip(img) for _ in range(self.augmentation_rate) ]
        if self.parameters[2]:
            transformed_images += [self.vflip(img) for _ in range(self.augmentation_rate) ]
        if self.parameters[3]:
            transformed_images += [self.crop(img) for _ in range(self.augmentation_rate) ]
        if self.parameters[4]:
            transformed_images += [self.jitter(img) for _ in range(self.augmentation_rate) ]
        if self.parameters[5]:
            transformed_images += [self.perspective_transformer(img) for _ in range(self.augmentation_rate) ]
        if self.parameters[6]:
            transformed_images += [self.rotater(img) for _ in range(self.augmentation_rate) ]
                
        # Compile results        
        results = []
        for transf_img in transformed_images:
            buff = io.BytesIO()
            transf_img.save(buff, format="JPEG")
            results.append(buff.getvalue().decode("latin1"))
             
        return results